In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import copy
import math
import plotly.express as px
import plotly.graph_objects as go
import plotly.figure_factory as ff
import plotly.subplots as sub
import dash
from dash import dcc, html, Output, Input, State
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, precision_recall_fscore_support, confusion_matrix, top_k_accuracy_score
from sklearn.model_selection import StratifiedKFold
import optuna
from optuna.trial import TrialState
#from optuna.integration import PyTorchIgnitePruningHandler

from functools import partial
import random
import os
import itertools
from PIL import Image
from tqdm.auto import tqdm

import albumentations as A
from albumentations.pytorch import ToTensorV2

import torch
import torchvision
import torch.optim as optim
from torch import nn
import torch.nn.functional as F
from torch.utils.data import DataLoader, Dataset, TensorDataset, Subset
from torchvision.models import vgg16_bn, resnet50, resnet18, efficientnet_b0, densenet121, ResNet50_Weights, ResNet18_Weights, VGG16_BN_Weights, DenseNet121_Weights, EfficientNet_B0_Weights
from torchvision.utils import make_grid, draw_bounding_boxes, draw_segmentation_masks, draw_keypoints
from torchvision import datasets
from torchvision.transforms import ToTensor, v2, ToPILImage
from torchvision.io import decode_image

# from ignite.engine import Events, create_supervised_trainer, create_supervised_evaluator
# from ignite.metrics import Accuracy, Loss, RunningAverage, ConfusionMatrix
# from ignite.handlers import ModelCheckpoint, EarlyStopping

from pathlib import Path
from torch.utils.tensorboard import SummaryWriter
from torch.amp import GradScaler, autocast

from transformers import ViTForImageClassification, ViTImageProcessor, AutoModelForImageClassification, AutoImageProcessor, Trainer, TrainingArguments
from huggingface_hub import snapshot_download, hf_hub_download
import socket
import json
import sys
import io
import base64

In [2]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [3]:
# Get the absolute path to the src folder
src_path = os.path.abspath(os.path.join(os.getcwd(), '..', 'src'))
sys.path.append(src_path)

In [4]:
from hyperopt import get_model, objective


In [5]:
from data_utils import FGVCAircraftDataset, get_datasets, get_loaders, get_raw
from models import CAPResNet, SEEffNet, LabelSmoothingCrossEntropy, FocalLoss
from aircraft_utils import train_one_epoch, evaluate, visualize_predictions

In [6]:
get_raw()

WindowsPath('C:/Users/chihp/Capstone/FGVCAircraft/fgvc-aircraft-2013b/data')

In [ ]:
study = optuna.create_study(study_name="Finest_FGVCAircraft_Variant", direction='maximize', 
                            pruner=optuna.pruners.MedianPruner(n_startup_trials=5, n_warmup_steps=5, interval_steps=3))
study.optimize(objective, timeout=None, n_jobs=1, n_trials=30, gc_after_trial=False, show_progress_bar=True)

print("Best trial:")
trial = study.best_trial
print(f"  Accuracy: {trial.value}")
print("  Params:")
for key, value in trial.params.items():
    print(f"    {key}: {value}")



[I 2025-08-06 21:54:48,711] A new study created in memory with name: Finest_FGVCAircraft_Variant


  0%|          | 0/30 [00:00<?, ?it/s]

Training:   0%|          | 0/105 [00:00<?, ?it/s]

Evaluation:   0%|          | 0/105 [00:00<?, ?it/s]

Epoch 1/20
Train Loss: 4.6095, Train Acc: 0.0117
Val Loss: 4.6086, Val Acc: 0.0123
✅ Model improved. Saving...


...

Evaluation:   0%|          | 0/105 [00:00<?, ?it/s]

Epoch 6/20
Train Loss: 4.5784, Train Acc: 0.0216
Val Loss: 4.5746, Val Acc: 0.0312
✅ Model improved. Saving...


Training:   0%|          | 0/105 [00:00<?, ?it/s]

[W 2025-08-06 22:04:29,673] Trial 0 failed with parameters: {'lr': 0.0067118816562944635, 'batch_size': 32, 'optimizer': 'SGD', 'weight_decay': 0.00033771497036264416, 'dropout_rate': 0.2459472342678516, 'backbone': 'EffNet_SE', 'scheduler': 'CosineAnnealingLR', 'criterion': 'LabelSmoothing', 'smoothing': 0.12102400695081754} because of the following error: KeyboardInterrupt().
Traceback (most recent call last):
  File "c:\Users\chihp\anaconda3\Lib\site-packages\optuna\study\_optimize.py", line 196, in _run_trial
    value_or_values = func(trial)
                      ^^^^^^^^^^^
  File "c:\Users\chihp\UMich\SIADS\GitHub\CV_aircraft_classifier_capstone_project\src\hyperopt.py", line 123, in objective
    train_loss, train_acc, _, _ = train_one_epoch(model, train_loader, criterion, optimizer, device, scaler)
                                  ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "c:\Users\chihp\UMich\SIADS\GitHub\CV_aircraft_classifier_capston

KeyboardInterrupt: 

In [ ]:
# Visualize optimization results
optuna.visualization.plot_optimization_history(study, target_name='Accuracy', error_bar=True).show()
optuna.visualization.plot_param_importances(study).show() # params=["x", "y"]

In [ ]:

# # Plot with custom labels
# plt.plot(range(len([t.value for t in study.trials if t.value is not None])), [t.value for t in study.trials if t.value is not None ], marker='o')
# plt.xlabel("Trial")
# plt.ylabel("Validation Accuracy")
# plt.title("Optimization History")
# plt.grid(True)

In [ ]:
optuna.visualization.plot_parallel_coordinate(study, target_name='Accuracy').show()
optuna.visualization.plot_slice(study, target_name='Accuracy').show()

In [ ]:
optuna.visualization.plot_contour(study, target_name = 'Accuracy').show()
optuna.visualization.plot_edf(study, target_name = 'Accuracy').show()

In [ ]:
optuna.visualization.plot_intermediate_values(study).show()
optuna.visualization.plot_rank(study, target_name = 'Accuracy').show()

In [ ]:
optuna.visualization.plot_timeline(study).show()